<a href="https://colab.research.google.com/github/RMoulla/SSL/blob/main/Fine_tuning_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Fine-tuning de BERT pour la classification de textes

Ce TP a pour objectif d'illustrer l'utilisation de BERT pour une tâche de classification de textes en utilisant le transfer learning. Nous utiliserons le dataset AG News qui contient des articles de presse à classifier en 4 catégories.


In [ ]:
!pip install torch transformers datasets tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## 1. Préparation de l'environnement

Commençons par importer les bibliothèques nécessaires et vérifier notre environnement d'exécution.


In [ ]:
# Cellule 1 : Imports et setup
import torch
from torch import nn
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Vérification du GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device utilisé : {device}")


## 2. Exploration et visualisation des données

Explorons maintenant le dataset AG News pour comprendre sa structure et sa distribution.


In [ ]:
# Chargement du dataset AG News
dataset = load_dataset("ag_news")
print("Structure du dataset :")
print(dataset)

# Affichage de quelques exemples
print("\nExemples du dataset :")
for i in range(3):
    print(f"\nExemple {i+1}:")
    print(f"Texte : {dataset['train'][i]['text']}")
    print(f"Label : {dataset['train'][i]['label']}")

# Statistiques sur les labels
labels = [x['label'] for x in dataset['train']]
unique, counts = np.unique(labels, return_counts=True)
print("\nDistribution des classes :")
for label, count in zip(unique, counts):
    print(f"Classe {label}: {count} exemples")

# Visualisation de la distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=labels)
plt.title("Distribution des classes dans le jeu d'entraînement")
plt.xlabel("Classe")
plt.ylabel("Nombre d'exemples")
plt.show()


## 3. Tokenization avec BERT

La tokenization est une étape cruciale dans le traitement du texte avec BERT. Nous allons explorer comment BERT découpe le texte en tokens.


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Exemple de tokenization
text = "Hello! How are you doing? 😊"
tokens = tokenizer.tokenize(text)
print(f"Texte original : {text}")
print(f"Tokens : {tokens}")

# Conversion en IDs
input_ids = tokenizer.encode(text, add_special_tokens=True)
print(f"\nInput IDs : {input_ids}")
print(f"Tokens décodés : {tokenizer.convert_ids_to_tokens(input_ids)}")

# Exemple complet de preprocessing
encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=32,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

print("\nSortie complete du tokenizer :")
print(f"Input IDs shape : {encoding['input_ids'].shape}")
print(f"Attention mask shape : {encoding['attention_mask'].shape}")
print(f"Attention mask : {encoding['attention_mask'][0]}")

## 4. Préparation des données

Création d'un Dataset personnalisé pour gérer nos données efficacement.

In [ ]:
class AGNewsDataset(Dataset):
    def __init__(self, split, tokenizer, max_len=128):
        self.dataset = load_dataset("ag_news")[split]
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = str(self.dataset[idx]['text'])
        label = self.dataset[idx]['label']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }



## 5. Architecture du modèle

Définition de notre modèle de classification basé sur BERT.
Points clés de l'architecture :
- Utilisation de BERT pré-entraîné comme base
- Ajout d'une couche de classification
- Gel des paramètres de BERT pour le transfer learning

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, num_classes=4, freeze_bert=True):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        ############# Code ##############


        #################################

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):

        ############# Code ##############


        #################################



# Création du modèle et analyse des paramètres
model = BERTClassifier(num_classes=4, freeze_bert=True)

# Analyse des paramètres
        ############# Code ##############


        #################################

print("Analyse des paramètres du modèle :")
print(f"Paramètres totaux : {total_params:,}")
print(f"Paramètres entraînables : {trainable_params:,}")
print(f"Paramètres gelés : {frozen_params:,}")

## 6. Fonctions d'entraînement et d'évaluation

Implémentation des fonctions nécessaires pour l'entraînement et l'évaluation du modèle.
Ces fonctions orchestrent :
- L'entraînement par batch
- Le calcul de la loss et de l'accuracy
- L'évaluation sur le jeu de test
- L'affichage de la progression

In [ ]:
def train_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    progress_bar = tqdm(data_loader, desc='Training')
    for batch in progress_bar:
        ############# Code ##############


        #################################

    return total_loss / len(data_loader), correct_predictions / total_predictions

def evaluate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

            _, predictions = torch.max(outputs, dim=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.shape[0]
            total_loss += loss.item()

    return (total_loss / len(data_loader),
            correct_predictions / total_predictions,
            all_predictions,
            all_labels)

## 7. Entraînement du modèle

In [ ]:
BATCH_SIZE = 32
EPOCHS = 3
LEARNING_RATE = 2e-5

# Création des dataloaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = AGNewsDataset('test', tokenizer, max_len=128)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Préparation du modèle
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# Suivi des métriques
history = {
    'train_loss': [], 'train_acc': [],
    'test_loss': [], 'test_acc': []
}

# Entraînement
for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch + 1}/{EPOCHS}')

    ############# Code ##############


    #################################

## 8. Analyse des résultats

Visualisation et analyse des performances du modèle.
Cette section nous permet de :
- Visualiser les courbes d'apprentissage
- Analyser la matrice de confusion
- Identifier les forces et faiblesses du modèle

In [ ]:
# Courbes d'apprentissage
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['test_loss'], label='Test')
plt.title('Loss au cours des époques')
plt.xlabel('Époque')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train')
plt.plot(history['test_acc'], label='Test')
plt.title('Accuracy au cours des époques')
plt.xlabel('Époque')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Matrice de confusion
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matrice de confusion')
plt.xlabel('Prédictions')
plt.ylabel('Vraies classes')
plt.show()


## 9. Test sur des exemples réels

Utilisation du modèle entraîné sur de nouveaux exemples.
Testez le modèle avec vos propres textes pour évaluer ses performances dans des conditions réelles.


In [ ]:
class_names = ['World', 'Sports', 'Business', 'Sci/Tech']
test_texts = [
    "NASA successfully launches new Mars rover to explore the red planet",
    "Manchester United wins dramatic match against Liverpool",
    "Apple stock reaches all-time high after strong quarterly earnings",
    "New study reveals breakthrough in quantum computing research"
]

############# Code ##############


#################################